In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import time
import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_FINE_TUNE = 10

BASE_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_trashnet_base"
PRUNED_MODEL_PATH = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_pruned_cek-arsitektur"

DATASET_DIR = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"

In [3]:
# DATA GENERATOR (RESCALE ONLY)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "train"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "val"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(DATASET_DIR, "test"),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 2001 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [4]:
# FUNGSI CEK UKURAN MODEL
def get_model_size_mb(model_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(model_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)

In [5]:
base_model = tf.keras.models.load_model(
    BASE_MODEL_PATH,
    compile=False
)

base_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

base_model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                61504     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 3058246 (11.67 MB)
Trainable params: 61894 (241.77 KB)
Non-trainable params: 2996352 (11.43 MB)
_________________________________________________________________


In [6]:
# CEK UKURAN BASE MODEL
base_model_size = get_model_size_mb(BASE_MODEL_PATH)
print(f"Ukuran Base Model      : {base_model_size:.2f} MB")

Ukuran Base Model      : 16.03 MB


In [7]:
def apply_pruning(model, initial_sparsity, final_sparsity, end_step):

    pruning_params = {
        "pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(
            initial_sparsity=initial_sparsity,
            final_sparsity=final_sparsity,
            begin_step=0,
            end_step=end_step
        )
    }

    def prune_layer(layer):
        if isinstance(layer, tf.keras.layers.DepthwiseConv2D):
            return layer
        if isinstance(layer, tf.keras.layers.Conv2D):
            return tfmot.sparsity.keras.prune_low_magnitude(layer, **pruning_params)
        return layer

    return tf.keras.models.clone_model(
        model,
        clone_function=prune_layer
    )

In [8]:
steps_per_epoch = np.ceil(train_data.samples / BATCH_SIZE)
end_step = int(steps_per_epoch * EPOCHS_FINE_TUNE)

In [9]:
pruned_model = apply_pruning(base_model, 0.0, 0.20, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
    EarlyStopping(patience=2, restore_best_weights=True)
]

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10


63/63 [==============================] - 67s 535ms/step - loss: 0.1675 - accuracy: 0.9660 - val_loss: 0.4276 - val_accuracy: 0.8488
Epoch 2/10
63/63 [==============================] - 11s 173ms/step - loss: 0.1512 - accuracy: 0.9710 - val_loss: 0.4196 - val_accuracy: 0.8568
Epoch 3/10
63/63 [==============================] - 11s 175ms/step - loss: 0.1408 - accuracy: 0.9740 - val_loss: 0.4178 - val_accuracy: 0.8594
Epoch 4/10
63/63 [==============================] - 11s 171ms/step - loss: 0.1304 - accuracy: 0.9780 - val_loss: 0.4156 - val_accuracy: 0.8568
Epoch 5/10
63/63 [==============================] - 11s 171ms/step - loss: 0.1203 - accuracy: 0.9815 - val_loss: 0.4048 - val_accuracy: 0.8568
Epoch 6/10
63/63 [==============================] - 11s 172ms/step - loss: 0.1118 - accuracy: 0.9835 - val_loss: 0.3998 - val_accuracy: 0.8621
Epoch 7/10
63/63 [==============================] - 11s 170ms/step - loss: 0.1033 - accuracy: 0.9835 - val_loss: 0.4131 - val_accuracy: 0.84

In [10]:
pruned_model = apply_pruning(pruned_model, 0.20, 0.30, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10
63/63 [==============================] - 32s 282ms/step - loss: 0.0936 - accuracy: 0.9875 - val_loss: 0.3987 - val_accuracy: 0.8541
Epoch 2/10
63/63 [==============================] - 11s 168ms/step - loss: 0.0838 - accuracy: 0.9910 - val_loss: 0.4031 - val_accuracy: 0.8594
Epoch 3/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0778 - accuracy: 0.9925 - val_loss: 0.3964 - val_accuracy: 0.8568
Epoch 4/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0723 - accuracy: 0.9955 - val_loss: 0.3963 - val_accuracy: 0.8568
Epoch 5/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0669 - accuracy: 0.9965 - val_loss: 0.4006 - val_accuracy: 0.8568
Epoch 6/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0616 - accuracy: 0.9970 - val_loss: 0.4036 - val_accuracy: 0.8594


In [11]:
pruned_model = apply_pruning(pruned_model, 0.30, 0.40, end_step)

pruned_model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

pruned_model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=callbacks
)

Epoch 1/10
63/63 [==============================] - 33s 293ms/step - loss: 0.0687 - accuracy: 0.9945 - val_loss: 0.3883 - val_accuracy: 0.8647
Epoch 2/10
63/63 [==============================] - 11s 167ms/step - loss: 0.0620 - accuracy: 0.9965 - val_loss: 0.3954 - val_accuracy: 0.8568
Epoch 3/10
63/63 [==============================] - 11s 170ms/step - loss: 0.0571 - accuracy: 0.9975 - val_loss: 0.3948 - val_accuracy: 0.8621


In [12]:
print("\nRingkasan Arsitektur Model Setelah Pruning (Sebelum Strip)")
pruned_model.summary()


Ringkasan Arsitektur Model Setelah Pruning (Sebelum Strip)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 64)                61504     
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 3058246 (11.67 MB)
Trainable params: 61894 (241.77 KB)
Non-trainable params: 2996352 (11.43 MB)
__________________

In [13]:
print("\nLayer yang Terkena Pruning Wrapper")
for layer in pruned_model.layers:
    if hasattr(layer, "pruning_step"):
        print(f"Layer dipruning: {layer.layer.name}")


Layer yang Terkena Pruning Wrapper


In [14]:
print("\nPersentase Bobot Nol per Layer (Sebelum Strip)")

for layer in pruned_model.layers:
    if hasattr(layer, "layer"):  # pruning wrapper
        weights = layer.layer.get_weights()
        if len(weights) > 0:
            w = weights[0]
            zero_ratio = np.sum(w == 0) / w.size * 100
            print(f"{layer.layer.name}: {zero_ratio:.2f}% bobot nol")


Persentase Bobot Nol per Layer (Sebelum Strip)


In [15]:
print("\nFilter Conv2D yang Sepenuhnya Nol")

for layer in pruned_model.layers:
    if hasattr(layer, "layer") and isinstance(layer.layer, tf.keras.layers.Conv2D):
        w = layer.layer.get_weights()[0]
        zero_filters = np.sum(np.all(w == 0, axis=(0, 1, 2)))
        total_filters = w.shape[-1]
        print(f"{layer.layer.name}: {zero_filters}/{total_filters} filter nol")


Filter Conv2D yang Sepenuhnya Nol


In [16]:
final_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

In [17]:
final_pruned_model.save(
    PRUNED_MODEL_PATH,
    save_format="tf"
)

print("Model pruning berhasil disimpan")

INFO:tensorflow:Assets written to: D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_pruned_cek-arsitektur\assets


INFO:tensorflow:Assets written to: D:/KULIAH/SEMESTER 7/Skripsi/Dataset/mobilenetv3Large_pruned_cek-arsitektur\assets


Model pruning berhasil disimpan


In [18]:
# CEK UKURAN MODEL SETELAH PRUNING
pruned_model_size = get_model_size_mb(PRUNED_MODEL_PATH)
print(f"Ukuran Pruned Model    : {pruned_model_size:.2f} MB")

compression_ratio = base_model_size / pruned_model_size
size_reduction = (1 - (pruned_model_size / base_model_size)) * 100

print(f"Compression Ratio     : {compression_ratio:.2f}x")
print(f"Size Reduction        : {size_reduction:.2f}%")

Ukuran Pruned Model    : 15.55 MB
Compression Ratio     : 1.03x
Size Reduction        : 2.94%


In [19]:
test_data.reset()

predictions = final_pruned_model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = test_data.classes

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

12/12 [==============================] - 11s 364ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89        61
           1       0.84      0.91      0.87        76
           2       0.90      0.85      0.88        62
           3       0.88      0.90      0.89        90
           4       0.79      0.85      0.82        72
           5       0.82      0.64      0.72        22

    accuracy                           0.86       383
   macro avg       0.86      0.83      0.84       383
weighted avg       0.86      0.86      0.86       383


Confusion Matrix:
[[52  0  0  8  0  1]
 [ 0 69  2  0  5  0]
 [ 0  3 53  3  3  0]
 [ 4  0  0 81  4  1]
 [ 0  7  3  0 61  1]
 [ 0  3  1  0  4 14]]


In [20]:
import time
import numpy as np

def benchmark_inference(model, data_gen, warmup=3, runs=10):
    """
    model      : stripped pruned model
    data_gen   : test_data (shuffle=False)
    warmup     : jumlah warmup run
    runs       : jumlah benchmark run
    """

    # Ambil 1 batch data
    x_batch, _ = next(data_gen)

    # Warm-up (penting untuk TensorFlow)
    for _ in range(warmup):
        _ = model.predict(x_batch, verbose=0)

    times = []

    for _ in range(runs):
        start = time.time()
        _ = model.predict(x_batch, verbose=0)
        end = time.time()
        times.append(end - start)

    avg_time = np.mean(times)
    per_image_time = avg_time / x_batch.shape[0]

    print("Inference Benchmark")
    print(f"Batch size         : {x_batch.shape[0]}")
    print(f"Avg batch time     : {avg_time:.4f} seconds")
    print(f"Avg per image time : {per_image_time:.6f} seconds")

    return avg_time, per_image_time

In [21]:
benchmark_inference(final_pruned_model, test_data)

Inference Benchmark
Batch size         : 32
Avg batch time     : 0.1857 seconds
Avg per image time : 0.005804 seconds


(0.18573572635650634, 0.005804241448640823)